# Week 07 — Neural Networks From Scratch

This notebook implements neural networks from first principles to deeply understand forward/backprop. You'll:
- Build a fully connected neural network in NumPy
- Implement backpropagation manually
- Perform gradient checking
- Experiment with initializations and activations

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_classification
from sklearn.model_selection import train_test_split
%matplotlib inline

np.random.seed(42)
print("Libraries imported!")

## 1. Build a Fully Connected Neural Network

Implement forward pass, loss, backward pass, and training loop from scratch.

In [ ]:
# Activation functions
def relu(x):
    return np.maximum(0, x)

def relu_grad(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_grad(x):
    s = sigmoid(x)
    return s * (1 - s)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Simple 2-layer neural network
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, init_std=0.01):
        """Initialize network parameters"""
        self.W1 = np.random.randn(input_size, hidden_size) * init_std
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * init_std
        self.b2 = np.zeros((1, output_size))
        
        # Cache for backprop
        self.cache = {}
    
    def forward(self, X):
        """Forward pass"""
        # Layer 1
        z1 = X @ self.W1 + self.b1
        a1 = relu(z1)
        
        # Layer 2
        z2 = a1 @ self.W2 + self.b2
        a2 = softmax(z2)
        
        # Cache for backward pass
        self.cache = {'X': X, 'z1': z1, 'a1': a1, 'z2': z2, 'a2': a2}
        return a2
    
    def backward(self, y_true):
        """Backward pass (backpropagation)"""
        m = y_true.shape[0]
        X = self.cache['X']
        a1 = self.cache['a1']
        a2 = self.cache['a2']
        z1 = self.cache['z1']
        
        # Output layer gradient
        dz2 = a2 - y_true  # Softmax + cross-entropy gradient
        dW2 = a1.T @ dz2 / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Hidden layer gradient
        da1 = dz2 @ self.W2.T
        dz1 = da1 * relu_grad(z1)
        dW1 = X.T @ dz1 / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        return {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2}
    
    def compute_loss(self, y_pred, y_true):
        """Cross-entropy loss"""
        m = y_true.shape[0]
        log_probs = -np.log(y_pred[range(m), y_true.argmax(axis=1)] + 1e-8)
        return np.mean(log_probs)
    
    def train_step(self, X, y, lr=0.01):
        """One training step"""
        # Forward
        y_pred = self.forward(X)
        loss = self.compute_loss(y_pred, y)
        
        # Backward
        grads = self.backward(y)
        
        # Update parameters
        self.W1 -= lr * grads['dW1']
        self.b1 -= lr * grads['db1']
        self.W2 -= lr * grads['dW2']
        self.b2 -= lr * grads['db2']
        
        return loss

print("Neural network class implemented!")

## 2. Train on a Toy Dataset

Test the network on a simple classification problem.

In [ ]:
# Generate toy dataset
X, y = make_moons(n_samples=300, noise=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# One-hot encode labels
def one_hot(y, n_classes=2):
    return np.eye(n_classes)[y]

y_train_oh = one_hot(y_train)
y_test_oh = one_hot(y_test)

# Initialize and train network
net = TwoLayerNet(input_size=2, hidden_size=10, output_size=2, init_std=0.1)

n_epochs = 1000
lr = 0.5
losses = []

for epoch in range(n_epochs):
    loss = net.train_step(X_train, y_train_oh, lr=lr)
    losses.append(loss)
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss:.4f}")

# Evaluate
y_pred_train = net.forward(X_train)
y_pred_test = net.forward(X_test)
train_acc = np.mean(y_pred_train.argmax(axis=1) == y_train_oh.argmax(axis=1))
test_acc = np.mean(y_pred_test.argmax(axis=1) == y_test_oh.argmax(axis=1))

print(f"\nFinal Training Accuracy: {train_acc*100:.2f}%")
print(f"Final Test Accuracy: {test_acc*100:.2f}%")

# Plot loss curve
plt.figure(figsize=(10, 5))
plt.plot(losses, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.grid(alpha=0.3)
plt.show()

## 3. Gradient Checking

Validate backprop implementation using numerical gradients.

In [ ]:
def gradient_check(net, X, y, epsilon=1e-5):
    """
    Numerical gradient checking
    
    Compare analytic gradients from backprop to numerical gradients
    from finite differences
    """
    # Get analytic gradients
    y_pred = net.forward(X)
    grads_analytic = net.backward(y)
    
    # Check each parameter
    params = {'W1': net.W1, 'b1': net.b1, 'W2': net.W2, 'b2': net.b2}
    
    for param_name, param in params.items():
        grad_analytic = grads_analytic[f'd{param_name}']
        grad_numerical = np.zeros_like(param)
        
        # Compute numerical gradient for a subset of parameters
        it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
        count = 0
        while not it.finished and count < 10:  # Check only first 10 for speed
            ix = it.multi_index
            old_value = param[ix]
            
            # f(x + eps)
            param[ix] = old_value + epsilon
            y_plus = net.forward(X)
            loss_plus = net.compute_loss(y_plus, y)
            
            # f(x - eps)
            param[ix] = old_value - epsilon
            y_minus = net.forward(X)
            loss_minus = net.compute_loss(y_minus, y)
            
            # Numerical gradient
            grad_numerical[ix] = (loss_plus - loss_minus) / (2 * epsilon)
            
            # Restore
            param[ix] = old_value
            it.iternext()
            count += 1
        
        # Compare
        diff = np.linalg.norm(grad_analytic.ravel()[:10] - grad_numerical.ravel()[:10])
        rel_error = diff / (np.linalg.norm(grad_analytic.ravel()[:10]) + np.linalg.norm(grad_numerical.ravel()[:10]) + 1e-8)
        
        print(f"{param_name}: relative error = {rel_error:.2e}")
        if rel_error < 1e-5:
            print(f"  ✓ Gradient check passed!")
        elif rel_error < 1e-3:
            print(f"  ~ Gradient check OK (might be acceptable)")
        else:
            print(f"  ✗ Gradient check FAILED!")

# Run gradient check on small batch
print("Running gradient check...\n")
net_check = TwoLayerNet(input_size=2, hidden_size=5, output_size=2, init_std=0.1)
X_small = X_train[:10]
y_small = y_train_oh[:10]
gradient_check(net_check, X_small, y_small)

## 4. Initialization Experiments

Compare different initialization strategies: small random, Xavier/Glorot, He.

In [ ]:
# Test different initializations
def train_with_init(init_type, n_epochs=500):
    """Train network with specific initialization"""
    if init_type == 'small':
        net = TwoLayerNet(2, 10, 2, init_std=0.01)
    elif init_type == 'xavier':
        net = TwoLayerNet(2, 10, 2, init_std=np.sqrt(2.0 / (2 + 10)))
    elif init_type == 'he':
        net = TwoLayerNet(2, 10, 2, init_std=np.sqrt(2.0 / 2))
    else:
        raise ValueError(f"Unknown init_type: {init_type}")
    
    losses = []
    for epoch in range(n_epochs):
        loss = net.train_step(X_train, y_train_oh, lr=0.5)
        losses.append(loss)
    
    return losses, net

# Compare initializations
init_types = ['small', 'xavier', 'he']
results = {}

for init_type in init_types:
    losses, net = train_with_init(init_type, n_epochs=500)
    results[init_type] = losses
    y_pred = net.forward(X_test)
    acc = np.mean(y_pred.argmax(axis=1) == y_test_oh.argmax(axis=1))
    print(f"{init_type:10s} - Final test accuracy: {acc*100:.2f}%")

# Plot comparison
plt.figure(figsize=(12, 5))
for init_type, losses in results.items():
    plt.plot(losses, label=init_type, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss: Initialization Comparison')
plt.legend()
plt.grid(alpha=0.3)
plt.yscale('log')
plt.show()

## Exercises for Further Practice

1. **Add More Layers**: Extend the network to 3+ layers
2. **Different Activations**: Implement tanh, leaky ReLU, and compare
3. **Batch Processing**: Add mini-batch training support
4. **Regularization**: Add L2 regularization to the loss and gradients
5. **Visualization**: Plot decision boundaries for different network configurations

## Deliverables Checklist

- [ ] Fully connected neural network implementation from scratch
- [ ] Gradient checking validation
- [ ] Initialization experiments with analysis
- [ ] Training plots and performance metrics

## Recommended Resources

- CS231n lecture notes on backpropagation
- Andrew Ng's notes on neural networks
- "Neural Networks and Deep Learning" by Michael Nielsen (online book)